# Assessing Data Quality

When data from DNA sequencing is produced, it is stored in a file format called FastQ. This file contains information not just about the DNA sequence itself (e.g. the nucleotide sequence) but also contains information about the quality of the DNA sequence. Sequencing DNA is not an error-free process. Like any measurement, there is a potential for error. In generating millions of bases of DNA sequence, tens or or hundreds of thousands of those bases may be mistaken. Fortunately, we can estimate the error; we do this using something called a [Phred](https://en.wikipedia.org/wiki/Phred_quality_score)
Now that we have the sequence data we want to be able to determine the quality of the sequencing reads. 




## FastQ file format

A fastQ file (file extension .fastq) is a file that contains perhaps hundreds of thousands or millions of individual sequence reads. Each sequence read is represented in 4 lines of the file. Here is an example:

```bash
@HWI-ST330:304:H045HADXX:1:1101:1111:61397
CACTTGTAAGGGCAGGCCCCCTTCACCCTCCCGCTCCTGGGGGANNNNNNNNNNANNNCGAGGCCCTGGGGTAGAGGGNNNNNNNNNNNNNNGATCTTGG
+
@?@DDDDDDHHH?GH:?FCBGGB@C?DBEGIIIIAEF;FCGGI#########################################################
```

Line |Description 
-----|----- 
1|Always begins with ‘@’ and then information about the read (e.g. it’s direction, the machine it was sequenced on, etc.)
2|The actual DNA sequence
3|Always begins with a ‘+’ and sometimes the same info in line 1
4|Has a string of characters which represent the quality scores; must have same number of characters as line 2

## Phred score in detail
A Phred score is a measure of uncertainty and error. The value of a [Phred](https://en.wikipedia.org/wiki/Phred_quality_score) score is an indication of how likely it is that a base reported in a sequencing read is in error. Here is the table describing the Phred score values:

Phred Quality Score|Probability of incorrect base call|Base call accuracy
-------------------|----------------------------------|------------------
10|1 in 10 |90%
20|1 in 100|99%
30|1 in 1000|99.9%
40|1 in 10,000|99.99%
50|1 in 100,000|99.999%
60|1 in 1,000,000|99.9999%

## Running and interpreting FastQC

[FastQC](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/) is one of several programs we can use to determine the quality of DNA sequence. A single fastQ file may have millions of individual sequencing reads, each with its own quality information (Phred core). The FastQC report generates graphs and descriptive statistics that allow us to get a sense of the overall quality of a file of sequencing data. Once we have that information, we can use it later on to trim and/or filter out low quality data.

**Per base sequence quality**
This report shows the average quality score across the length of all reads. Poor quality at the beginning or end of the reads may suggest settings for trimming.

**Per sequence quality scores**
This report indicates how individual reads of a given quality score are distributed in your sequence file. Ideally, most reads will have a high average quality score. Populations of lower average-scored reads can be removed by downstream filtering.

**Adapter Content**
This report indicates the presence of sequencing adapters. If adapters are detected, you will need to remove them in downstream cleaning.


## Installing FastQC

We will install fastqc using the [conda](https://docs.conda.io/en/latest/) package manager. We are doing this in the Jupyter notebook but this and all other commands will work just about identically on any linux terminal/machine. 

### Important!!! - complete each of the numbered steps

1. We will search for the tool we want to install

We will use the `conda search` command and the channel (`-c`) flag to search [bioconda](https://bioconda.github.io/)

In [1]:
conda search fastqc -c bioconda

Loading channels: done
# Name                       Version           Build  Channel             
fastqc                        0.10.1               0  bioconda            
fastqc                        0.10.1               1  bioconda            
fastqc                        0.11.2               1  bioconda            
fastqc                        0.11.2      pl5.22.0_0  bioconda            
fastqc                        0.11.3               0  bioconda            
fastqc                        0.11.3               1  bioconda            
fastqc                        0.11.4               0  bioconda            
fastqc                        0.11.4               1  bioconda            
fastqc                        0.11.4               2  bioconda            
fastqc                        0.11.5               1  bioconda            
fastqc                        0.11.5               4  bioconda            
fastqc                        0.11.5      hdfd78af_5  bioconda            
fa

2. Create a conda enviornment

Conda uses something called "enviornments" which are essentially isolated configurations on our computer where we can included all the needed compatible tools and exlude other tools which are unnesessary or would have conflicts with our desired tool. We will use the `-y` option to install without prompting the user for input, the `--name` option to name the enviornment for the tool. We will enforce versioning (`tool==version`) so that we know what version of a tool was used to do an analysis should we wish to repeat the analysis. 

**Tip**: Use the latest version where possible, but if you get an error with dependancies, using a lower version may help. Some tools may never be installed successfully using conda, but we will face those when we have too. 


In [2]:
conda create -y --name fastqc fastqc==0.11.9 -c bioconda

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 22.11.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/exouser/.conda/envs/fastqc

  added / updated specs:
    - fastqc==0.11.9


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _openmp_mutex-5.1          |            1_gnu          21 KB
    expat-2.4.9                |       h6a678d5_0         156 KB
    fastqc-0.11.9              |       hdfd78af_1         9.7 MB  bioconda
    font-ttf-dejavu-sans-mono-2.37|       hd3eb1b0_0         335 KB
    fontconfig-2.14.1          |       h52c9d5c_1         281 KB
    freetype-2.12.1            |       h4a9f257_0         626 KB
    gdbm-1.18                  |       hd4cb3f1_4         194 KB
    glib-2.69.1                |       he621ea3_2  

3. We will use the `conda init` command so that conda can be configured for this shell

In [3]:
conda init

no change     /opt/anaconda3/condabin/conda
no change     /opt/anaconda3/bin/conda
no change     /opt/anaconda3/bin/conda-env
no change     /opt/anaconda3/bin/activate
no change     /opt/anaconda3/bin/deactivate
no change     /opt/anaconda3/etc/profile.d/conda.sh
no change     /opt/anaconda3/etc/fish/conf.d/conda.fish
no change     /opt/anaconda3/shell/condabin/Conda.psm1
no change     /opt/anaconda3/shell/condabin/conda-hook.ps1
no change     /opt/anaconda3/lib/python3.9/site-packages/xontrib/conda.xsh
no change     /opt/anaconda3/etc/profile.d/conda.csh
modified      /home/exouser/.bashrc

==> For changes to take effect, close and re-open your current shell. <==



4. **DON'T SKIP**: We need to restart the computer's [kernal](https://en.wikipedia.org/wiki/Kernel_(operating_system)). Go to the **Kernal** menu and choose **Restart Kernal**

5. Finally, we can activate the conda enviornment (created with the name used for the environment). When you run the next cell it should return the name of the environment.  

In [1]:
conda activate fastqc

(fastqc) 

: 1

## Connecting to our project and dataset folders

We have to check that this notebook is running in the expected directory, and that you have a project folder. When you run the following command, your working directory should be `/home/exouser`

1. Check our working directory

In [2]:
pwd

/mnt/ceph/notebooks
(fastqc) 

: 1

**(Optional)**: If the `pwd` command did not give you `/home/exouser` run the following command, otherwise **skip**. Please also keep in mind - you **should not** be running this notebook in `/mnt/ceph/notebooks`, so ask for help if you get this result.

In [3]:
cd /home/exouser # only run this if needed

(fastqc) 

: 1

2. Inspect the `project` folder. 

You should also have a project folder which is linked to your home folder on the shared drive. Running the following command should generate something like:

`lrwxrwxrwx 1 exouser exouser 19 Dec 30 21:27 jwilliams -> /mnt/ceph/YOURUSERNAME`

In [6]:
ls -lh project --color=never

total 0
lrwxrwxrwx 1 exouser exouser 19 Dec 30 21:27 jwilliams -> /mnt/ceph/jwilliams
(fastqc) 

: 1

**(Optional)**: If you don't have a `project` folder linked to your folder on the shared drive, run the command below. Remeber to change `YOURUSERNAME` to the name of your folder. 

In [ ]:
mkdir -p project && ln -s /mnt/ceph/YOURUSERNAME project # run only if needed, fix YOURUSERNAME

3. Connect to and inspect the folder of FASTQ reads. 

Let's also make sure the project folder has a link to the FASTQ reads we will use and also create a place for the results of your FastQC analysis. 

The next command will create a link in your project folder to a location on the shared drive where we have placed all of the Chamecrista FASTQ reads. 

In [7]:
ln -s /mnt/ceph/chamecrista_sup_fastq project/chamecrista_fastq

(fastqc) 

: 1

4. List the reads

When we look in the newly created `chamecrista_fastq` folder we should see a collection of reads we can analyze. 

In [16]:
ls -Rlh project/chamecrista_fastq/ 

project/chamecrista_fastq/:
total 23G
-rw-rw-r-- 1 exouser exouser  70M Dec 30 18:52 10000_called_reads.fastq
-rw-rw-r-- 1 exouser exouser  33M Dec 30 19:55 10000_called_reads.fastq.gz
-rw-rw-r-- 1 exouser exouser 6.7M Dec 30 18:52 1000_called_reads.fastq
-rw-rw-r-- 1 exouser exouser 3.2M Dec 30 19:55 1000_called_reads.fastq.gz
-rw-rw-r-- 1 exouser exouser 152K Dec 30 18:51 100_called_reads.fastq
-rw-rw-r-- 1 exouser exouser  76K Dec 30 19:54 100_called_reads.fastq.gz
-rw-rw-r-- 1 exouser exouser  15G Dec 30 18:45 called_reads.fastq
-rw-rw-r-- 1 exouser exouser 7.1G Dec 30 19:19 called_reads.fastq.gz
(fastqc) 

: 1

The following summarizes what we have:

- `called_reads.fastq` : All of the reads done during the 2022 sequencing project
- `100_called_reads.fastq`: A subset of the first 100 reads in `called_reads.fastq`
- `1000_called_reads.fastq`: A subset of the first 1000 reads in `called_reads.fastq`
- `10000_called_reads.fastq`: A subset of the first 10000 reads in `called_reads.fastq`

The versions of the above withe the `.gz` file extension are compressed versions of the above list. 

5. Create an output folder for the `fastqc` analysis. 

Finally, let's create a place in our project directory to store the output of our `FastQC` analysis. 

In [26]:
mkdir -p project/fastqc_results

(fastqc) 

: 1

## Running Fast QC

First, we will be run FastQC on a small subset of the sequence reads. In this exercise, the reads are in the `project/chamecrista_fastq` folder. We will run on a sample that has 10000 fastq files from the experiment (`1000_called_reads.fastq.gz`). 

**Tip**: When using commands or searching for files, the tab key will help you autocomplete (and help ensure the files and commands you think you have are actually accessible).

1. Run fastqc on the `1000_called_reads.fastq.gz` file located in concat_fastq

**Command breakdown**

- `fastqc` - The name of the program
- `~/project/chamecrista_fastq/1000_called_reads.fastq.gz` - The location and name of the file we want to analyze
- `-t 16` the number of CPU threads to use (*Note* generally, the higher this number the faster your analysis. However this number can't be higher than the actual number of CPUs you have available to do the work
- `-o ~/project/fastqc_results` - The location to write the output results

In [32]:
fastqc ~/project/chamecrista_fastq/1000_called_reads.fastq.gz -t 16 -o ~/project/fastqc_results

Started analysis of 10000_called_reads.fastq.gz
Approx 5% complete for 10000_called_reads.fastq.gz
Approx 15% complete for 10000_called_reads.fastq.gz
Approx 30% complete for 10000_called_reads.fastq.gz
Approx 40% complete for 10000_called_reads.fastq.gz
Approx 50% complete for 10000_called_reads.fastq.gz
Approx 55% complete for 10000_called_reads.fastq.gz
Approx 65% complete for 10000_called_reads.fastq.gz
Approx 75% complete for 10000_called_reads.fastq.gz
Approx 85% complete for 10000_called_reads.fastq.gz
Approx 100% complete for 10000_called_reads.fastq.gz
Analysis complete for 10000_called_reads.fastq.gz
(fastqc) 

: 1

2. In the file browser (left of your screen) navigate into the `~/project/fastqc_results` folder to locate the following outputs:

- There are two outputs from the fastqc command.

         1. An .html file (than can be viewed as a webpage)
         2. A .zip file that contains individual images and reports

3. Use the file browser to take a look at the results we can double click on the .html file and navigate through the different reports.

# Questions

The documentation for each fastqc report is located in the [fastqc help](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/Help/3%20Analysis%20Modules/)


- How many sequences were analyzed; what were their lengths?

-  Is the per base sequence quality more or less the same across the length of the sequence? Are there locations of the sequence that tend to have higher or lower Phred scores?


- Are there over represented sequences? Can they be identified ?

# Challenges

1. Repeat this analysis for the sample of 10000 reads (~/project/chamecrista_fastq/10000_called_reads.fastq.gz) and then the entire data set (~/project/chamecrista_fastq/called_reads.fastq.gz). Save your work in this notebook.

**Tip**: Run fastqc with the `-t` option so that it can use all available cpus. For example `fastqc -t 16 reads.fastq.gz` would use 16 cpus. Check to make sure the number of CPUs matches the number you launched with using cyverse. You can check the number using the command `cat /proc/cpuinfo` - add "1" to the last processor listed since the count starts at 0. See also `fastqc --help`